### This is my attempt to do shap... I hope

In [8]:
import pandas as pd
import numpy as np
import pandas_gbq
import yfinance as yf
import datetime

import shap

#### Lets get some model output data

In [9]:
project_id = 'irlts-317602'
# Full table path in gcp bigquery irlts-317602.2018.10eps_default_training
dataset_name = '2017'
table_name = '10eps_default'
r_query = """
SELECT 
    data.* 
FROM 
    `irlts-317602.2018.10eps_default_training` as data 
WHERE 
    data.Episode = '8'
"""
query_result = pandas_gbq.read_gbq(r_query,project_id=project_id, progress_bar_type=None)

In [10]:
query_result.Date = query_result.Date.astype(np.datetime64)

In [11]:
output_data = query_result.set_index('Date').sort_index()

#### Lets get some feature data

In [12]:
last_output_date = output_data.index.max()
first_output_date = output_data.index.min()
feature_df = yf.download('VOO',start=first_output_date,end=last_output_date, interval='1d')

[*********************100%***********************]  1 of 1 completed


In [13]:
feature_df_subset = feature_df[['Adj Close','Volume']]
# feature_df_subset.index

#### Lets join these two dfs (hoping this goes well)

In [21]:
final_data = query_result.join(feature_df_subset,on="Date",how = 'inner') #join the two datasets
final_data.rename(columns={f'{j}':f'{j.lower()}'for i,j in enumerate(final_data.columns)},inplace = True) #fix columns because this is bothering me
final_data.rename(columns={'adj close':'adj_close'},inplace=True)


In [22]:
final_data.columns

Index(['date', 'hold', 'buy', 'sell', 'choice', 'episode', 'adj_close',
       'volume'],
      dtype='object')

In [23]:
prediction = final_data.choice
x_train = final_data[['adj_close','volume']]

In [50]:
prediction =prediction.astype('int')

#### More Data Wrangling

#### An attempt to do SHAP

In [126]:
M = len(x_train.columns)
x = x_train.values[-1]
reference = np.zeros(M) #adj_close,volume
phi = kernel_shap(f, x, reference, M)
base_value = phi[-1]
shap_values = phi[:-1]

In [130]:
import scipy.special
import numpy as np
import itertools

def powerset(iterable):
    s = list(iterable)
    return itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(len(s)+1))

def shapley_kernel(M,s):
    if s == 0 or s == M:
        return 10000
    return (M-1)/(scipy.special.binom(M,s)*s*(M-s))

def f(X):
    """
    just pull the y value corresponding to x inputs
    """
    # np.random.seed(0)
    # beta = np.random.rand(X.shape[-1])
    # return np.dot(X,beta) + 10
    return prediction.values[-1]

def kernel_shap(f, x, reference, M):
    """
    Wrapper for the above functions
    """
    X = np.zeros((2**M,M+1))
    X[:,-1] = 1
    weights = np.zeros(2**M) # initialize weights
    V = np.zeros((2**M,M))
    for i in range(2**M):
        V[i,:] = reference

    ws = {}
    for i,s in enumerate(powerset(range(M))):
        s = list(s)
        V[i,s] = x[s]
        X[i,s] = 1
        ws[len(s)] = ws.get(len(s), 0) + shapley_kernel(M,len(s))
        weights[i] = shapley_kernel(M,len(s))
    y = f(V)
    tmp = np.linalg.inv(np.dot(np.dot(X.T, np.diag(weights)), X))
    return np.dot(tmp, np.dot(np.dot(X.T, np.diag(weights)), y))

print("  reference =", reference)
print("          x =", x)
print("shap_values =", shap_values)
print(" base_value =", base_value)
print("   sum(phi) =", np.sum(phi))
print("       model output =", f(x))

  reference = [0. 0.]
          x = [2.3440921e+02 9.0370000e+06]
shap_values = [[0. 0. 0. 0.]
 [0. 0. 0. 0.]]
 base_value = [0. 0. 0. 0.]
   sum(phi) = 0.0
       model output = 2


In [108]:
X = np.zeros((2**M,M+1))
# print(X)
X[:,-1] = 1
# print(X)
weights = np.zeros(2**M)
V = np.zeros((2**M,M))
for i in range(2**M):
    V[i,:] = reference
print(V)
ws = {}
for i,s in enumerate(powerset(range(M))):
    s = list(s)
    print(i,s)
    print(x[s])
    # V[i,s] = x[s].flatten()
    # X[i,s] = 1
    # ws[len(s)] = ws.get(len(s), 0) + shapley_kernel(M,len(s))
    # weights[i] = shapley_kernel(M,len(s))
# y = f(V)
# tmp = np.linalg.inv(np.dot(np.dot(X.T, np.diag(weights)), X))

[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
0 []
[]
1 [0]
[[2.43352722e+02 2.85730000e+06]]
2 [1]
[[2.41671005e+02 5.14500000e+06]]
3 [0, 1]
[[2.43352722e+02 2.85730000e+06]
 [2.41671005e+02 5.14500000e+06]]
